In [55]:
import requests
from bs4 import BeautifulSoup
from lxml import etree
import json
import re
import multiprocessing
# import xlwt
import time
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import prettytable as pt

In [ ]:
'''读取城市code.json'''

# json文件 转 Python数据类型
with open("D:/03workplace/pachong/ex04_12306/citycode.json",
          'r', encoding='utf-8') as f:
    city_data = json.load(f)

In [ ]:
city_data['上海']

In [ ]:
'''输入出发到达城市，时间'''

from_station = city_data[input('departure city')]
to_station = city_data[input('arrive city')]
train_date = input('date (format: 2022-05-01)')

# print(from_station, to_station, train_date)

'''根据输入-更改url'''
aim_link = 'https://kyfw.12306.cn/otn/leftTicket/queryZ?'\
        'leftTicketDTO.train_date={}&leftTicketDTO.from_station={}&'\
        'leftTicketDTO.to_station={}&purpose_codes=ADULT'.format(train_date, from_station, to_station)
# print(aim_link)

header = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                        'AppleWebKit/537.36 (KHTML, like Gecko) '
                        'Chrome/92.0.4515.107 Safari/537.36 Edg/92.0.902.55',
          'Cookie': '_uab_collina=166383369752571822153616; JSESSIONID=DF7EF7E7BD4C9AE3DB630A223EBB7ABC; BIGipServerotn=418382346.50210.0000; RAIL_EXPIRATION=1664130073550; RAIL_DEVICEID=JM23qN7Whm3pkS5TjxWBqKegm7ebeDHn6CLWwR8SKmY-1u45fy5Y83v9rSLPWMuisqG_4MoopusAslMgnXjdHnGz7mkuaYngFv8oY8Oh9geHA0lBgvtlT84_2g3TLOEzxao9U1IXHX0wsi8P1OX5RuOqai9arg4t; BIGipServerpool_passport=149160458.50215.0000; guidesStatus=off; highContrastMode=defaltMode; cursorStatus=off; fo=mr4xxi63eitk7a6rIMBl60T0R7mb2JQ3aPSMIO3c_3RFjJi7fu7SsaRmhT5N0XWyDiaJk-Ar5oUKnthIhYQqfJPOppWyXzkXpriBsBsqD7QiXru-tRLga3i80SQwvSCRUuZMh-YuAxo7udCPbWpAG4fbzaWYVGjDeTtk3CbK7mQkm1L_rckwNJrP_Zk; route=495c805987d0f5c8c84b14f60212447d; ai_user=yn5FsvthZPkgVt4UK8uDxu|2022-09-22T08:01:36.281Z; _jc_save_fromStation=%u5E7F%u5DDE%2CGZQ; _jc_save_toDate=2022-09-22; _jc_save_wfdc_flag=dc; _jc_save_fromDate=2022-09-30; _jc_save_toStation=%u676D%u5DDE%2CHZH'}

response = requests.get(aim_link, headers=header)
print(response.status_code)

In [49]:
""" 获取内容 """

# change to Chinese
response.encoding = response.apparent_encoding

'''M1'''
response.text

'{"httpstatus":200,"data":{"result":["xsOqM3huEj9hhaVj9VSh6I%2F6VeZi8orTv0RTi%2FPOPKZrJ9aBOA7bxBovbZwFRXx3D5Qq60yFcR3C%0An9HoF5lxRJ%2BZMtzOVQyaYUHJBjlr84A2sUO%2B%2FpoWsmoibtnjG1T%2Fs1y%2BYeifOz5CocE18keDkw9y%0AMuDm5yznF2OCoUrW6cUvBDDXSI8dS4dLFUB8kDN%2BiYvBqdjZOahQSwvr0FT7YYf3eoE7OgBgiRDV%0AiJipSQNB%2F7iy%2FnCpDcIkvwCS6AJht7XNZh1%2B2Ju06%2B0klmWPBz0%2BU2U1sR1snA2JSiVQMMBUGwSy%0AkUgSv%2BjgiN14X5wS|预订|630000Z13500|Z135|GZQ|WAR|GZQ|XAY|06:55|07:42|24:47|Y|TmB4Z7%2Fa41jfJcss5cHpNkmgS14v1%2FEmz7eImplUs9c19SRTQhMJq4p2kuQ%3D|20220927|3|Q6|01|14|1|0||||无|||无||11|无|||||304010W0|3411|0|1||3041400011407240000010243000001024303000|0|||||1|#0#0#0#z||","nC%2FGPyk1XkSgIqnXasXLAJ0csVWvOVydC91tpG%2FPIibnLP2g405g7zE7VYbcUzA7hG6J8s5jh3YX%0AH0AIWftVXPMFiiCKiwt5oTfDkMygqvTl4gAj1JJwTnuatiEfFOOx741l%2Bsvh6Iegv61Uj5lwWiQb%0ACdANDTR%2F8PHVb4qikRHf5Gi3NrmICRj5dRfSU8gv5cV9WBRtULPfNVH5tW3fEUmJkszRBV7VVgch%0ATtLmiLmRagaDaz2OhQz48DgzPMDnJ0IE1AJrS5mEFfcJP6hV6BOiKuxSZZV6ty8ZYVRtZ9VQPesZ%0A9GS3wVFgkxY%3D|预订|6c0000G8340

In [50]:
'''M2'''

response.json()
# type(response.json()) # dict

{'httpstatus': 200,
 'data': {'result': ['xsOqM3huEj9hhaVj9VSh6I%2F6VeZi8orTv0RTi%2FPOPKZrJ9aBOA7bxBovbZwFRXx3D5Qq60yFcR3C%0An9HoF5lxRJ%2BZMtzOVQyaYUHJBjlr84A2sUO%2B%2FpoWsmoibtnjG1T%2Fs1y%2BYeifOz5CocE18keDkw9y%0AMuDm5yznF2OCoUrW6cUvBDDXSI8dS4dLFUB8kDN%2BiYvBqdjZOahQSwvr0FT7YYf3eoE7OgBgiRDV%0AiJipSQNB%2F7iy%2FnCpDcIkvwCS6AJht7XNZh1%2B2Ju06%2B0klmWPBz0%2BU2U1sR1snA2JSiVQMMBUGwSy%0AkUgSv%2BjgiN14X5wS|预订|630000Z13500|Z135|GZQ|WAR|GZQ|XAY|06:55|07:42|24:47|Y|TmB4Z7%2Fa41jfJcss5cHpNkmgS14v1%2FEmz7eImplUs9c19SRTQhMJq4p2kuQ%3D|20220927|3|Q6|01|14|1|0||||无|||无||11|无|||||304010W0|3411|0|1||3041400011407240000010243000001024303000|0|||||1|#0#0#0#z||',
   'nC%2FGPyk1XkSgIqnXasXLAJ0csVWvOVydC91tpG%2FPIibnLP2g405g7zE7VYbcUzA7hG6J8s5jh3YX%0AH0AIWftVXPMFiiCKiwt5oTfDkMygqvTl4gAj1JJwTnuatiEfFOOx741l%2Bsvh6Iegv61Uj5lwWiQb%0ACdANDTR%2F8PHVb4qikRHf5Gi3NrmICRj5dRfSU8gv5cV9WBRtULPfNVH5tW3fEUmJkszRBV7VVgch%0ATtLmiLmRagaDaz2OhQz48DgzPMDnJ0IE1AJrS5mEFfcJP6hV6BOiKuxSZZV6ty8ZYVRtZ9VQPesZ%0A9GS3wVFgkxY%3D|预订|6c0

In [51]:
''' M2等效 '''
data = json.loads(response.text)
data
# type(data) # dict

{'httpstatus': 200,
 'data': {'result': ['xsOqM3huEj9hhaVj9VSh6I%2F6VeZi8orTv0RTi%2FPOPKZrJ9aBOA7bxBovbZwFRXx3D5Qq60yFcR3C%0An9HoF5lxRJ%2BZMtzOVQyaYUHJBjlr84A2sUO%2B%2FpoWsmoibtnjG1T%2Fs1y%2BYeifOz5CocE18keDkw9y%0AMuDm5yznF2OCoUrW6cUvBDDXSI8dS4dLFUB8kDN%2BiYvBqdjZOahQSwvr0FT7YYf3eoE7OgBgiRDV%0AiJipSQNB%2F7iy%2FnCpDcIkvwCS6AJht7XNZh1%2B2Ju06%2B0klmWPBz0%2BU2U1sR1snA2JSiVQMMBUGwSy%0AkUgSv%2BjgiN14X5wS|预订|630000Z13500|Z135|GZQ|WAR|GZQ|XAY|06:55|07:42|24:47|Y|TmB4Z7%2Fa41jfJcss5cHpNkmgS14v1%2FEmz7eImplUs9c19SRTQhMJq4p2kuQ%3D|20220927|3|Q6|01|14|1|0||||无|||无||11|无|||||304010W0|3411|0|1||3041400011407240000010243000001024303000|0|||||1|#0#0#0#z||',
   'nC%2FGPyk1XkSgIqnXasXLAJ0csVWvOVydC91tpG%2FPIibnLP2g405g7zE7VYbcUzA7hG6J8s5jh3YX%0AH0AIWftVXPMFiiCKiwt5oTfDkMygqvTl4gAj1JJwTnuatiEfFOOx741l%2Bsvh6Iegv61Uj5lwWiQb%0ACdANDTR%2F8PHVb4qikRHf5Gi3NrmICRj5dRfSU8gv5cV9WBRtULPfNVH5tW3fEUmJkszRBV7VVgch%0ATtLmiLmRagaDaz2OhQz48DgzPMDnJ0IE1AJrS5mEFfcJP6hV6BOiKuxSZZV6ty8ZYVRtZ9VQPesZ%0A9GS3wVFgkxY%3D|预订|6c0

In [ ]:
''' 内容解析 '''

In [56]:
'''pretty table'''
tb = pt.PrettyTable()
tb.field_names = [
    '车次',
    '发车时间',
    '到达时间',
    '车程',
    '特等',
    '一等',
    '二等',
    '软卧',
    '硬卧',
    '硬座',
    '无座'
]

'''解析'''
for ele in response.json()['data']['result']:
    info = ele.split('|')
    # print([*enumerate(ele.split('|'))])
    t_num = info[3]

    start_time = info[8]
    end_time = info[9]
    used_time = info[10]

    topGrade = info[32]
    first_class = info[31]
    second_class = info[30]

    soft_sleeper = info[23]
    hard_sleeper = info[28]

    # soft_seat = info[]
    hard_seat = info[29]
    no_seat = info[26]

    dic = {'车次': t_num,
           '发车时间': start_time, '到达时间': end_time, '车程': used_time,
           '特等': topGrade, '一等': first_class, '二等': second_class,
           '软卧': soft_sleeper, '硬卧': hard_sleeper,
           '硬座': hard_seat, '无座': no_seat
           }  #'软座': soft_seat,
    print(dic)

    tb.add_row([
        t_num,
        start_time, end_time, used_time,
        topGrade, first_class, second_class,
        soft_sleeper, hard_sleeper,
        hard_seat, no_seat
    ])

{'车次': 'Z135', '发车时间': '06:55', '到达时间': '07:42', '车程': '24:47', '特等': '', '一等': '', '二等': '', '软卧': '无', '硬卧': '11', '硬座': '无', '无座': '无'}
{'车次': 'G834', '发车时间': '07:03', '到达时间': '16:07', '车程': '09:04', '特等': '8', '一等': '有', '二等': '有', '软卧': '', '硬卧': '', '硬座': '', '无座': ''}
{'车次': 'G878', '发车时间': '08:42', '到达时间': '18:05', '车程': '09:23', '特等': '2', '一等': '13', '二等': '有', '软卧': '', '硬卧': '', '硬座': '', '无座': ''}
{'车次': 'G814', '发车时间': '08:55', '到达时间': '16:15', '车程': '07:20', '特等': '9', '一等': '有', '二等': '有', '软卧': '', '硬卧': '', '硬座': '', '无座': ''}
{'车次': 'K1348', '发车时间': '10:15', '到达时间': '13:22', '车程': '27:07', '特等': '', '一等': '', '二等': '', '软卧': '7', '硬卧': '有', '硬座': '有', '无座': '无'}
{'车次': 'G822', '发车时间': '10:15', '到达时间': '19:22', '车程': '09:07', '特等': '无', '一等': '9', '二等': '4', '软卧': '', '硬卧': '', '硬座': '', '无座': ''}
{'车次': 'Z230', '发车时间': '10:55', '到达时间': '09:05', '车程': '22:10', '特等': '', '一等': '', '二等': '', '软卧': '', '硬卧': '无', '硬座': '无', '无座': ''}
{'车次': 'G838', '发车时间': '11:25', '到达时间

In [57]:
tb

车次,发车时间,到达时间,车程,特等,一等,二等,软卧,硬卧,硬座,无座
Z135,06:55,07:42,24:47,,,,无,11,无,无
G834,07:03,16:07,09:04,8,有,有,,,,
G878,08:42,18:05,09:23,2,13,有,,,,
G814,08:55,16:15,07:20,9,有,有,,,,
K1348,10:15,13:22,27:07,,,,7,有,有,无
G822,10:15,19:22,09:07,无,9,4,,,,
Z230,10:55,09:05,22:10,,,,,无,无,
G838,11:25,20:13,08:48,无,无,有,,,,
G842,12:23,21:16,08:53,1,无,有,,,,
G830,13:29,22:36,09:07,无,4,10,,,,


登录，验证，抢票

- ctrl + F -- 搜索想要的信息，快速定位Link

In [22]:
htmls = etree.HTML(response.text)
htmls

<Element html at 0x20316bb0580>

In [ ]:
'''M2'''
bs = BeautifulSoup(response.text, 'lxml')
print(bs)